In [1]:
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd
import numpy as np

In [2]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Ilostat")

In [3]:
df = pd.read_csv("Ilostat_paises_selecionados_genero.csv", sep=";", thousands=".", decimal=',')

In [6]:
## Subindo para GBQ

client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao') # verificar a categoria do dado

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pais            52 non-null     object
 1   fonte_pesquisa  52 non-null     object
 2   genero          52 non-null     object
 3   quantidade      52 non-null     int64 
 4   setor           52 non-null     object
 5   ano_pesquisa    52 non-null     int64 
dtypes: int64(2), object(4)
memory usage: 2.6+ KB


In [9]:
df=df.rename(columns={'País':'pais'})

In [11]:
schema=[
 bigquery.SchemaField('pais','STRING',description='Nome do país selecionado'),
 bigquery.SchemaField('fonte_pesquisa','STRING',description='De qual pesquisa nacional foi extraído aquele dado.'),
bigquery.SchemaField('genero','STRING',description='Gênero da pessoa'),
bigquery.SchemaField('quantidade','INTEGER',description='total de pessoas que trabalham'),
bigquery.SchemaField('setor','STRING',description='se o setor é público ou privado'),
 bigquery.SchemaField('ano_pesquisa','Float',description='Ano de coleta da informação'),
 
 ]

In [12]:
table_ref = dataset_ref.table('Ilostat_paises_selecionados_genero') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 